# Neural Machine Translation Example

In [1]:
# Install the package via PyPI
!pip install headliner

     |████████████████████████████████| 1.5MB 7.7MB/s 
     |████████████████████████████████| 266kB 37.0MB/s 
     |████████████████████████████████| 86.3MB 1.2MB/s 
     |████████████████████████████████| 3.8MB 27.3MB/s 
     |████████████████████████████████| 450kB 35.6MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449906 sha256=34d26861b3c473168e648393b0e53b671532ba37a9fd360448f5d981a4bfb3cf
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=a293791976da2a26e06e4f1c54ba9e66d58b0ee198d92b0dcc5b92df741cfc2f
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=fa4113ac17731648ba5d079ea2e5e620a4901af24b087cd9b3ce5047f6b696b8
  Stored in directory: /root/.cache/pip/wheels/5c/2

In [2]:
# Download the German-English sentence pairs
!wget http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip
!head deu.txt

--2019-10-01 14:57:17--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4541707 (4.3M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   4.33M  2.44MB/s    in 1.8s    

2019-10-01 14:57:19 (2.44 MB/s) - ‘deu-eng.zip’ saved [4541707/4541707]

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              
Hi.	Hallo!
Hi.	Grüß Gott!
Run!	Lauf!
Wow!	Potzdonner!
Wow!	Donnerwetter!
Fire!	Feuer!
Help!	Hilfe!
Help!	Zu Hülf!
Stop!	Stopp!
Wait!	Warte!


In [0]:
# Create the dataset
import io

def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]
    return zip(*word_pairs)

eng, ger = create_dataset('deu.txt', 500)
data = list(zip(ger, eng))

In [0]:
# Split the dataset into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1)

In [5]:
# Define the model and train it
from headliner.trainer import Trainer
from headliner.model.summarizer_attention import SummarizerAttention

summarizer = SummarizerAttention(lstm_size=64, embedding_size=24)
trainer = Trainer(batch_size=32, steps_per_epoch=100, steps_to_log=20, model_save_path='/tmp/summarizer')
trainer.train(summarizer, train, num_epochs=10, val_data=test)

training a bare model, initializing preprocessing...
vocab encoder: 383, vocab decoder: 203, start training loop...
finished iterating over dataset, total batches: 14
epoch 0, batch 20, logs: {'loss': 4.0605316162109375}
finished iterating over dataset, total batches: 28
epoch 0, batch 40, logs: {'loss': 3.1279962062835693}
finished iterating over dataset, total batches: 42
finished iterating over dataset, total batches: 56
epoch 0, batch 60, logs: {'loss': 2.5084290504455566}
finished iterating over dataset, total batches: 70
epoch 0, batch 80, logs: {'loss': 2.934246778488159}
finished iterating over dataset, total batches: 84
finished iterating over dataset, total batches: 98
epoch 0, batch 100, logs: {'loss': 2.871776580810547}

(input) Ich bin auf. 
(target) I'm up. 
(prediction) . . <end>


(input) Ich bin neu. 
(target) I'm new. 
(prediction) . . <end>


(input) Rettet Tom! 
(target) Save Tom. 
(prediction) . . <end>


(input) Ich benutze es. 
(target) I use it. 
(prediction) . 

In [6]:
# Do some prediction
summarizer.predict('Hi.')

'come on . <end>'